LIBRARY:

In [237]:
import numpy as np
import pandas as pd

FUNCTION:

In [238]:
def size_to_float(y):
    if (y != y.rstrip('M')):
        return float(y.rstrip('M'))*1024
    if(y != y.rstrip('k')):
        return float(y.rstrip('k'))    
    if(y == 'Varies with device'):
        return np.nan
    
def installs_to_float(y): 
    if(y == 'Free'):
        return np.nan
    return float(y.rstrip('+'))

def price_to_float(y):
    if(y=='Everyone'):
        return np.nan
    return float(y.lstrip('$'))

def word_in_review(translated_review):
    return len(translated_review.split())


this cell contains project tasks, from number 1 to number 5, and other fix to the dataSources in order to run next project tasks

In [286]:
#Starting from the Google Play Store dataset
gpApp = pd.read_csv("/home/man01/Desktop/FondamentiCS/googleplaystore.csv")
gpReview = pd.read_csv("/home/man01/Desktop/FondamentiCS/googleplaystore_user_reviews.csv")

# apply kylobyte function on gpstore.Size column, removing 'M' or 'k' from the string value and transforming MEGABYTE to KILOBYTE
#1 
#Also converting gpApp.Size from string to float
gpApp.Size = gpApp.Size.map(size_to_float)

#removing '+' and ',' from the string value, replace Free value with np.nan
#2
#Converting Installs value from string to float
gpApp['Installs'] = gpApp['Installs'].str.replace(",", "")
gpApp.Installs = gpApp.Installs.map(installs_to_float)

#3Transform “Varies with device” into a missing value
gpApp = gpApp.replace('Varies with device', np.nan)

#removing all character after a ' 'character
#4
#Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)
#fixing 4.0.4W case
gpApp['Android Ver'] = gpApp['Android Ver'].str.split(' ').str[0]
gpApp["Android Ver"] = gpApp["Android Ver"].str.replace("W", "").str.strip()

#5
#removing duplicates
oldSize = gpApp.count()['App']
#gpApp[gpApp.duplicated()].size
gpApp = gpApp.drop_duplicates()
gpApp = gpApp.reset_index(drop=True)
newSize = gpApp.count()['App']

# For each category, compute the number of apps
# For each category, compute the average rating
gpApp.Rating[gpApp.Rating>5] =np.nan
gpApp["Category"] = gpApp["Category"].str.replace("1.9", "").str.strip()

#Create two dataframes: one for the genres and one bridging apps and genres. 
#So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, 
#once for Art & Design, once for Creativity
gpApp["Genres"] = gpApp["Genres"].str.replace("February 11, 2018", "").str.strip()

#For each genre, create a new column of the original dataframe. 
#The new columns must have boolean values (True if the app has a given genre)
gpApp['Genres'] = gpApp['Genres'].str.split(";")

#For each app, compute the approximate income, obtain as a product of number of installs and price.
gpApp.Price=gpApp.Price.map(price_to_float)

#For each app, compute its longest review



/usr/lib/python3/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [240]:
#6 VV
#For each category, compute the number of apps
#7 VV
#For each category, compute the average rating
for cat in gpApp.Category.unique():
    gpCat = gpApp[gpApp.Category==cat]
    print("Category : ",cat," app n° : ", gpCat.Category.count()," Rating avg : ", gpCat.Rating.mean())


Category :  ART_AND_DESIGN  app n° :  65  Rating avg :  4.358064516129032
Category :  AUTO_AND_VEHICLES  app n° :  85  Rating avg :  4.19041095890411
Category :  BEAUTY  app n° :  53  Rating avg :  4.2785714285714285
Category :  BOOKS_AND_REFERENCE  app n° :  230  Rating avg :  4.3474576271186445
Category :  BUSINESS  app n° :  427  Rating avg :  4.102592592592592
Category :  COMICS  app n° :  60  Rating avg :  4.155172413793103
Category :  COMMUNICATION  app n° :  366  Rating avg :  4.151465798045603
Category :  DATING  app n° :  196  Rating avg :  3.9716981132075473
Category :  EDUCATION  app n° :  130  Rating avg :  4.375968992248062
Category :  ENTERTAINMENT  app n° :  111  Rating avg :  4.136036036036035
Category :  EVENTS  app n° :  64  Rating avg :  4.435555555555556
Category :  FINANCE  app n° :  360  Rating avg :  4.127444794952682
Category :  FOOD_AND_DRINK  app n° :  124  Rating avg :  4.164150943396227
Category :  HEALTH_AND_FITNESS  app n° :  306  Rating avg :  4.261450381

In [241]:
#8 VV
#Create two dataframes: one for the genres and one bridging apps and genres. 
#So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, 
#once for Art & Design, once for Creativity
gen = pd.DataFrame(gpApp['Genres'].str[0].unique(), columns={'Genre'})
ag1 = pd.DataFrame({'App':gpApp.App , 
              'Genre':gpApp['Genres'].str[0]})
ag2 = pd.DataFrame({'App':gpApp.App , 
              'Genre':gpApp['Genres'].str[1]})
appGen = pd.concat([ag1,ag2[ag2.Genre.notnull()]])

print(appGen[appGen.App=='Pixel Draw - Number Art Coloring Book'])


                                     App         Genre
4  Pixel Draw - Number Art Coloring Book  Art & Design
4  Pixel Draw - Number Art Coloring Book    Creativity


In [242]:
#9 VV
#For each genre, create a new column of the original dataframe. 
#The new columns must have boolean values (True if the app has a given genre)
dGen = pd.get_dummies(gpApp['Genres'].apply(pd.Series).stack()).sum(level=0)
gpApp = pd.concat([gpApp, dGen], axis=1)
gpApp


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19456.0,10000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14336.0,500000.0,Free,0.0,Everyone,"[Art & Design, Pretend Play]",...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8908.8,5000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25600.0,50000000.0,Free,0.0,Teen,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2867.2,100000.0,Free,0.0,Everyone,"[Art & Design, Creativity]",...,0,0,0,0,0,0,0,0,0,0
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5734.4,50000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19456.0,50000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29696.0,1000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33792.0,1000000.0,Free,0.0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3174.4,10000.0,Free,0.0,Everyone,"[Art & Design, Creativity]",...,0,0,0,0,0,0,0,0,0,0


In [243]:
#10 VV
#For each genre, compute the average rating. What is the genre with highest average?
#maxAverage = maxR
for g in gen.Genre.unique():
    gpGen = gpApp[gpApp.Genres.apply(lambda x: g in x)]
    print("Genre Rating : ", g,gpGen.Rating.mean())
    if g == gen.Genre.unique()[0]:
        maxR = [g,gpGen.Rating.mean()]
    if gpGen.Rating.mean()>maxR[1]:
        maxR = (g,gpGen.Rating.mean())

print("highest avg Genre : ", maxR)

Genre Rating :  Art & Design 4.349230769230769
Genre Rating :  Auto & Vehicles 4.19041095890411
Genre Rating :  Beauty 4.2785714285714285
Genre Rating :  Books & Reference 4.345810055865923
Genre Rating :  Business 4.102592592592592
Genre Rating :  Comics 4.155172413793103
Genre Rating :  Communication 4.151623376623377
Genre Rating :  Dating 3.9716981132075473
Genre Rating :  Education 4.299484536082475
Genre Rating :  Entertainment 4.101301115241636
Genre Rating :  Events 4.435555555555556
Genre Rating :  Finance 4.127444794952682
Genre Rating :  Food & Drink 4.164150943396227
Genre Rating :  Health & Fitness 4.261742424242424
Genre Rating :  House & Home 4.164705882352941
Genre Rating :  Libraries & Demo 4.178461538461539
Genre Rating :  Lifestyle 4.0967320261437905
Genre Rating :  Adventure 4.219101123595506
Genre Rating :  Arcade 4.305909090909091
Genre Rating :  Casual 4.181176470588235
Genre Rating :  Card 4.102083333333334
Genre Rating :  Action 4.280769230769231
Genre Rating :

In [244]:
#11 VV
#For each app, compute the approximate income, obtain as a product of number of installs and price.
appIncome=gpApp.Price*gpApp.Installs 
for inc in range(appIncome.count()):
    print(inc, gpApp['App'][inc], appIncome[inc])

0 Photo Editor & Candy Camera & Grid & ScrapBook 0.0
1 Coloring book moana 0.0
2 U Launcher Lite – FREE Live Cool Themes, Hide Apps 0.0
3 Sketch - Draw & Paint 0.0
4 Pixel Draw - Number Art Coloring Book 0.0
5 Paper flowers instructions 0.0
6 Smoke Effect Photo Maker - Smoke Editor 0.0
7 Infinite Painter 0.0
8 Garden Coloring Book 0.0
9 Kids Paint Free - Drawing Fun 0.0
10 Text on Photo - Fonteee 0.0
11 Name Art Photo Editor - Focus n Filters 0.0
12 Tattoo Name On My Photo Editor 0.0
13 Mandala Coloring Book 0.0
14 3D Color Pixel by Number - Sandbox Art Coloring 0.0
15 Learn To Draw Kawaii Characters 0.0
16 Photo Designer - Write your name with shapes 0.0
17 350 Diy Room Decor Ideas 0.0
18 FlipaClip - Cartoon animation 0.0
19 ibis Paint X 0.0
20 Logo Maker - Small Business 0.0
21 Boys Photo Editor - Six Pack & Men's Suit 0.0
22 Superheroes Wallpapers | 4K Backgrounds 0.0
23 Mcqueen Coloring pages 0.0
24 HD Mickey Minnie Wallpapers 0.0
25 Harley Quinn wallpapers HD 0.0
26 Colorfit - Dra

405 Call Control - Call Blocker 0.0
406 True Contact - Real Caller ID 0.0
407 Video Caller Id 0.0
408 Sync.ME – Caller ID & Block 0.0
409 Burner - Free Phone Number 0.0
410 Truecaller: Caller ID, SMS spam blocking & Dialer 0.0
411 Caller ID + 0.0
412 Gmail 0.0
413 Yahoo Mail – Stay Organized 0.0
414 K-9 Mail 0.0
415 myMail – Email for Hotmail, Gmail and Outlook Mail 0.0
416 Email TypeApp - Mail App 0.0
417 All Email Providers 0.0
418 Newton Mail - Email App for Gmail, Outlook, IMAP 0.0
419 GO Notifier 0.0
420 Mail.Ru - Email App 0.0
421 Mail1Click - Secure Mail 0.0
422 Daum Mail - Next Mail 0.0
423 mail.com mail 0.0
424 SolMail - All-in-One email app 0.0
425 Hangouts 0.0
426 Viber Messenger 0.0
427 Vonage Mobile® Call Video Text 0.0
428 JusTalk - Free Video Calls and Fun Video Chat 0.0
429 Moco+ - Chat, Meet People 39900.0
430 Calculator 6990.0
431 Truth or Dare Pro 74.5
432 Private Dating, Hide App- Blue for PrivacyHider 299.0
433 Ad Blocker for SayHi 399.0
434 AMBW Dating App: Asian 

738 SoloLearn: Learn to Code for Free 0.0
739 Kids Learn Languages by Mondly 0.0
740 Blinkist - Nonfiction Books 0.0
741 Sago Mini Hat Maker 3990.0
742 Fuzzy Numbers: Pre-K Number Foundation 5990.0
743 Toca Life: City 1995000.0
744 Toca Life: Hospital 399000.0
745 Netflix 0.0
746 Complete Spanish Movies 0.0
747 Pluto TV - It’s Free TV 0.0
748 Tubi TV - Free Movies & TV 0.0
749 YouTube Kids 0.0
750 Mobile TV 0.0
751 TV+ 0.0
752 Digital TV 0.0
753 Motorola Spotlight Player™ 0.0
754 Vigo Lite 0.0
755 Google Play Games 0.0
756 Hotstar 0.0
757 Peers.TV: broadcast TV channels First, Match TV, TNT ... 0.0
758 The green alien dance 0.0
759 Spectrum TV 0.0
760 H TV 0.0
761 StarTimes - Live International Champions Cup 0.0
762 Cinematic Cinematic 0.0
763 MEGOGO - Cinema and TV 0.0
764 Talking Angela 0.0
765 DStv Now 0.0
766 ivi - movies and TV shows in HD 0.0
767 Radio Javan 0.0
768 Viki: Asian TV Dramas & Movies 0.0
769 Talking Ginger 2 0.0
770 Girly Lock Screen Wallpaper with Quotes 0.0
771 No.

1071 Easy Healthy Recipes 0.0
1072 Allrecipes Dinner Spinner 0.0
1073 My CookBook Pro (Ad Free) 34900.0
1074 Paprika Recipe Manager 249500.0
1075 Yummly Recipes & Shopping List 0.0
1076 Kitchen Stories - Recipes & Cooking 0.0
1077 Cookpad 0.0
1078 BigOven Recipes, Meal Planner, Grocery List & More 0.0
1079 Postmates Food Delivery: Order Eats & Alcohol 0.0
1080 Instacart: Grocery Delivery 0.0
1081 OpenTable: Restaurants Near Me 0.0
1082 Foursquare City Guide 0.0
1083 Zomato - Restaurant Finder and Food Delivery App 0.0
1084 Munchery: Chef Crafted Fresh Food Delivered 0.0
1085 delivery.com: Order Food, Alcohol & Laundry 0.0
1086 Eat24 Food Delivery & Takeout 0.0
1087 BeyondMenu Food Delivery 0.0
1088 EatStreet Food Delivery App 0.0
1089 Pizza Hut 0.0
1090 Caviar - Food Delivery 0.0
1091 DoorDash - Food Delivery 0.0
1092 Chick-fil-A 0.0
1093 Uber Eats: Local Food Delivery 0.0
1094 Seamless Food Delivery/Takeout 0.0
1095 Talabat: Food Delivery 0.0
1096 TheFork - Restaurants booking and spe

1571 Episode - Choose Your Story 0.0
1572 Hello Stars 0.0
1573 Run Sausage Run! 0.0
1574 Tank Stars 0.0
1575 Castle Clash: Heroes of the Empire US 0.0
1576 Hole.io 0.0
1577 Helix Jump 0.0
1578 Mini Golf King - Multiplayer Game 0.0
1579 PUBG MOBILE 0.0
1580 Flip the Gun - Simulator Game 0.0
1581 Mad Skills BMX 2 0.0
1582 MMX Hill Dash 2 – Offroad Truck, Car & Bike Racing 0.0
1583 Word Link 0.0
1584 Last Day on Earth: Survival 0.0
1585 MARVEL Strike Force 0.0
1586 Partymasters - Fun Idle Game 0.0
1587 Harry Potter: Hogwarts Mystery 0.0
1588 Offroad Outlaws 0.0
1589 Grim Soul: Dark Fantasy Survival 0.0
1590 DRAGON BALL LEGENDS 0.0
1591 Disney Heroes: Battle Mode 0.0
1592 Homescapes 0.0
1593 New YAHTZEE® With Buddies Dice Game 0.0
1594 Word Crossy - A crossword game 0.0
1595 MLB TAP SPORTS BASEBALL 2018 0.0
1596 Hero Hunters 0.0
1597 Cooking Madness - A Chef's Restaurant Games 0.0
1598 Ice Crush 2018 - A new Puzzle Matching Adventure 0.0
1599 TerraGenesis - Space Colony 0.0
1600 Merge Drag

...

7735 Movie DB 0.0
7736 dB Sound Level Meter 0.0
7737 DB Sound Meter: Measure Noise Level- Decibel Meter 0.0
7738 DB Welt - Die Zeitung der DB 0.0
7739 Check DB Benchmark 0.0
7740 ACCDB MDB DB Manager Pro - Editor for MS Access 4495.0
7741 Scale Models DB 149.5
7742 DB Train Simulator 0.0
7743 DB Tools 0.0
7744 dB Meter - measure sound & noise level in Decibel 0.0
7745 DB LINK RGB 0.0
7746 DB BAHN 0.0
7747 DB Manager 0.0
7748 10,000 Quotes DB (Premium) 495.0
7749 Guide for DB Xenoverse 0.0
7750 DB Transport 0.0
7751 DB Stay Connected 0.0
7752 DB FahrtProfi 0.0
7753 DB TOS - Pocket Helper 0.0
7754 ReDNAKET DB Normalization Tool 9.9
7755 Guide for DB Xenoverse 2 0.0
7756 DB Customer Connect 0.0
7757 Patron DB 0.0
7758 DB Event App 0.0
7759 DB HOME 0.0
7760 Bowers & Wilkins DB Subwoofers 0.0
7761 Guide for IMS DB 0.0
7762 DB Busradar NRW 0.0
7763 DB Pickles 0.0
7764 DB for Fallout Shelter 0.0
7765 LC-DB 34.900000000000006
7766 dB Sound Meter 0.0
7767 DB for Hustle Castle 0.0
7768 DB Ultra 

8568 DX Simulation for Double Dx Henshin Belt 2018 0.0
8569 Mircules DX Cluster Lite 0.0
8570 DX Simulation Belt for Decade henshin 0.0
8571 Chronolink DX Lite 0.0
8572 Shoot! DX - lights for FREE 0.0
8573 Bubble Shooter DX AdFree 149.5
8574 Human Dx 0.0
8575 Santa's Monster Shootout DX 99.5
8576 DX Glow - Clock Widget 49.5
8577 iCluster - The DX-Cluster database 9.9
8578 MBU DX Cluster 0.0
8579 Dress Up RagazzA13 DX 99.0
8580 Cloud DX Connected Health 0.0
8581 DX Simulation for X-aid Dx Belt 0.0
8582 Unstoppaball DX 74.5
8583 Differential Dx 1995.0
8584 Sokoban Land DX 29900.000000000004
8585 Multiple Sclerosis Dx & Mgmt. 0.0
8586 Dx Ludo 0.0
8587 My Dy Dice - 3D Dice Roller 0.0
8588 Dy So Exam 0.0
8589 edp re:dy 0.0
8590 Dr. D. Y. Patil Vidyapeeth GBS 0.0
8591 edp re:dy smarthome 0.0
8592 KPS CHAUHAN DY/DX 0.0
8593 DY Fitness 0.0
8594 dy n fly 0.0
8595 Cnady Selfie : You Can Dy Snap Camera 0.0
8596 junainfo.dy.fi 0.0
8597 DY TECHNICAL GYAN 0.0
8598 JONACK (D Story Teller) 0.0
8599 Am

9250 Operate Now: Hospital 0.0
9251 Multi Surgery ER Emergency Hospital : Doctor Game 0.0
9252 ER Surgery Simulator 0.0
9253 Dentist Surgery ER Emergency Doctor Hospital Games 0.0
9254 Eye Transplant : ER Emergency Hospital 0.0
9255 Lifeguard Beach Rescue ER Emergency Hospital Games 0.0
9256 Hospital ER Emergency Heart Surgery: Doctor Games 0.0
9257 Superhero Doctor 2 -ER Surgery 0.0
9258 ER 0.0
9259 Noot-Noot-er! 0.0
9260 Doctor X - ER On Wheels 0.0
9261 Foot Surgery Hospital Simulator: ER Doctor Games 0.0
9262 ER Heart Surgery - Emergency Simulator Game 0.0
9263 Pregnant Mom ER Emergency Doctor Hospital Games 0.0
9264 Doctr – ER wait times in Canada 0.0
9265 Plastic Surgery Surgeon Simulator Er Doctor Games 0.0
9266 Shoulder Surgery ER Emergency Doctor Game 0.0
9267 Superhero Doctor 3 ER Surgery 0.0
9268 Mom Doctor ER Emergency Family Game 0.0
9269 ER Blood Draw Doctor & Surgeon 0.0
9270 ER Emergency Hospital - Brain, Knee, Eye Surgery 0.0
9271 Super Doctor 1-Pregnant Mom ER Surgery 

9567 Money Manager Ex for Android 0.0
9568 Advanced EX for NISSAN 24950.0
9569 Advanced EX for FIAT 24950.0
9570 Advanced EX for RENAULT 24950.0
9571 EX File Explorer File Manage Pro 99.5
9572 Aurum Blade EX 0.0
9573 SCI-Ex 0.0
9574 Papyrus Ex 0.0
9575 GLASS GO Launcher EX Theme 0.0
9576 Moonlight GO Weather EX 0.0
9577 My Ex Girlfriend Comes Back 0.0
9578 An Elite Warrior Ex 99.0
9579 Gods Wars Ex : Vampire 0.0
9580 BakaReader EX 0.0
9581 Snes9x EX+ 0.0
9582 X your Ex - Break Up Treatment 0.0
9583 Ex Quotes 0.0
9584 GO Contacts EX Black & Grey 495.0
9585 CYANOGEN GO Launcher EX Theme 0.0
9586 Ex Service Taxis 0.0
9587 Rejoin Your Ex 0.0
9588 S.O.L : Stone of Life EX 0.0
9589 How To Get Your Ex Back Fast 0.0
9590 ExDialer PRO Key 399000.0
9591 Volume Control Ex 149.0
9592 Virtual Dice EX 0.0
9593 Black Theme Go Launcher EX 0.0
9594 EY Events 0.0
9595 EY GlobalOne 0.0
9596 EY Team Connect 0.0
9597 EY eMentor 0.0
9598 EY Conferences 0.0
9599 EY Africa 0.0
9600 EY TaxChat 0.0
9601 EY Expe

9901 Family Guy The Quest for Stuff 0.0
9902 Ambulance Simulator 3D 0.0
9903 Leghe Fantagazzetta 0.0
9904 Police Car Driving Sim 0.0
9905 Toy Attack 0.0
9906 Field Goal Tournament 0.0
9907 Ghost In Photo 0.0
9908 Disney Princess Palace Pets 0.0
9909 Police VAZ LADA Simulator 0.0
9910 Scary Video Maker 0.0
9911 SnowMobile Parking Adventure 0.0
9912 PBS KIDS Games 0.0
9913 Zombie Sniper 3D III 0.0
9914 Family Guy- Another Freakin' Mobile Game 0.0
9915 FH WiFiCam 0.0
9916 HD Themes Volvo FH Trucks 0.0
9917 Familial Hypercholesterolaemia 0.0
9918 Familial Hypercholesterolaemia Handbook 0.0
9919 Themes Volvo FH 16 Trucks 0.0
9920 FH Calculator 0.0
9921 Top Themes Volvo FH Trucks 0.0
9922 Top Puzzles Volvo FH Trucks 0.0
9923 Wallpapers Volvo FH 16 Trucks 0.0
9924 Puzzles Volvo FH 16 Trucks 0.0
9925 Jigsaw Volvo FH 16 Trucks 0.0
9926 Shoot Hunter-Gun Killer 0.0
9927 studentsLife by FH Kärnten 0.0
9928 FH® Cost Lookup / FH® CCSalud 0.0
9929 FH Kufstein App 0.0
9930 FH App 0.0
9931 HD Wallpaper

In [287]:
#12 VV
#For each app, compute its minimum and maximum Sentiment_polarity
for rap in gpReview.App.unique():
    gpRap = gpReview[gpReview.App==rap]
    print("App : ", rap," Min SentimentP : ", gpRap.Sentiment_Polarity.min()," Max SentimentP : ", gpRap.Sentiment_Polarity.max())

App :  10 Best Foods for You  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室  Min SentimentP :  -0.11250000000000004  Max SentimentP :  0.91
App :  11st  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  1800 Contacts - Lens Store  Min SentimentP :  -0.3  Max SentimentP :  0.8385416666666667
App :  1LINE – One Line with One Touch  Min SentimentP :  -0.825  Max SentimentP :  1.0
App :  2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  21-Day Meditation Experience  Min SentimentP :  -0.265625  Max SentimentP :  0.5875
App :  2Date Dating App, Love and matching  Min SentimentP :  -0.6458333333333334  Max SentimentP :  1.0
App :  2GIS: directory & navigator  Min SentimentP :  -0.375  Max SentimentP :  1.0
App :  2RedBeans  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  2ndLine - Second Phone Number  Min SentimentP :  -0.78125  Max SentimentP :  1.0
App :  30 Day Fitness Challenge - Workout 

App :  Animal Jam - Play Wild!  Min SentimentP :  nan  Max SentimentP :  nan
App :  Animal Planet GO  Min SentimentP :  -0.2916666666666667  Max SentimentP :  0.8
App :  Animated Photo Editor  Min SentimentP :  -0.8  Max SentimentP :  0.91
App :  Anime Avatar Creator: Make Your Own Avatar  Min SentimentP :  -0.8  Max SentimentP :  0.96875
App :  Anime Manga Coloring Book  Min SentimentP :  -0.2916666666666667  Max SentimentP :  1.0
App :  Anthem Anywhere  Min SentimentP :  -1.0  Max SentimentP :  0.6145833333333334
App :  Anthem BC Anywhere  Min SentimentP :  -1.0  Max SentimentP :  0.6
App :  Any.do: To-do list, Calendar, Reminders & Planner  Min SentimentP :  -0.4  Max SentimentP :  1.0
App :  Apartment Decorating Ideas  Min SentimentP :  0.245  Max SentimentP :  0.245
App :  Apartment List: Housing, Apt, and Property Rentals  Min SentimentP :  -0.27901785714285715  Max SentimentP :  1.0
App :  Apartment, Home Rental Search: Realtor.com Rentals  Min SentimentP :  -1.0  Max SentimentP

App :  BestCam Selfie-selfie, beauty camera, photo editor  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  BetterMe: Weight Loss Workouts  Min SentimentP :  -0.3  Max SentimentP :  0.8687500000000001
App :  Betterment  Min SentimentP :  -0.6999999999999998  Max SentimentP :  1.0
App :  BeyondMenu Food Delivery  Min SentimentP :  -0.2  Max SentimentP :  1.0
App :  BeyondPod Podcast Manager  Min SentimentP :  -0.19047619047619047  Max SentimentP :  0.39333333333333337
App :  Bible  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  Big Days - Events Countdown  Min SentimentP :  -0.5  Max SentimentP :  0.7666666666666666
App :  BigOven Recipes, Meal Planner, Grocery List & More  Min SentimentP :  -0.06785714285714288  Max SentimentP :  1.0
App :  BiggerCity: Chat for gay bears, chubs & chasers  Min SentimentP :  -1.0  Max SentimentP :  0.9
App :  Bike Computer - GPS Cycling Tracker  Min SentimentP :  -0.09666666666666666  Max SentimentP :  0.8
App :  Binaural Beats Meditation  Min 

App :  Call Free – Free Call  Min SentimentP :  nan  Max SentimentP :  nan
App :  Call of Duty:Black Ops Zombies  Min SentimentP :  -0.2125  Max SentimentP :  -0.08135850694444442
App :  CallApp: Caller ID, Blocker & Phone Call Recorder  Min SentimentP :  0.3907407407407408  Max SentimentP :  0.3907407407407408
App :  Caller ID +  Min SentimentP :  0.35  Max SentimentP :  0.35
App :  Calls & Text by Mo+  Min SentimentP :  0.0  Max SentimentP :  0.7
App :  Calls Blacklist - Call Blocker  Min SentimentP :  -0.09345238095238097  Max SentimentP :  0.4320000000000001
App :  Calm - Meditate, Sleep, Relax  Min SentimentP :  -0.25  Max SentimentP :  1.0
App :  Calorie Counter & Diet Tracker  Min SentimentP :  -0.8  Max SentimentP :  0.8
App :  Calorie Counter - EasyFit free  Min SentimentP :  -0.425  Max SentimentP :  0.9
App :  Calorie Counter - Macros  Min SentimentP :  -0.225  Max SentimentP :  1.0
App :  Calorie Counter - MyFitnessPal  Min SentimentP :  -0.5  Max SentimentP :  1.0
App :  C

App :  Colorfy: Coloring Book for Adults - Free  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  Coloring & Learn  Min SentimentP :  -0.40454545454545454  Max SentimentP :  1.0
App :  Coloring book moana  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  Comedy Central  Min SentimentP :  -0.2357142857142857  Max SentimentP :  1.0
App :  Common Core  Min SentimentP :  -0.8  Max SentimentP :  0.35
App :  CompTIA A+ Pocket Prep  Min SentimentP :  nan  Max SentimentP :  nan
App :  CompTIA Exam Training  Min SentimentP :  nan  Max SentimentP :  nan
App :  Compass  Min SentimentP :  nan  Max SentimentP :  nan
App :  Comptia A+ 220-901 & 220-902  Min SentimentP :  0.0  Max SentimentP :  1.0
App :  ConnectLine  Min SentimentP :  -1.0  Max SentimentP :  0.7999999999999999
App :  Contacts  Min SentimentP :  -1.0  Max SentimentP :  0.7
App :  Contacts+  Min SentimentP :  -0.25  Max SentimentP :  1.0
App :  Content Transfer  Min SentimentP :  -0.48828125  Max SentimentP :  1.0
App :  Conve

App :  Docs To Go™ Free Office Suite  Min SentimentP :  -0.6999999999999998  Max SentimentP :  1.0
App :  Doctor On Demand  Min SentimentP :  -0.7000000000000001  Max SentimentP :  1.0
App :  Doctor Pets  Min SentimentP :  -1.0  Max SentimentP :  0.8
App :  Dog Licks Screen Wallpaper  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  Dog Run - Pet Dog Simulator  Min SentimentP :  -0.7  Max SentimentP :  1.0
App :  Dog Sim Online: Raise a Family  Min SentimentP :  -0.030965909090909068  Max SentimentP :  0.5716666666666667
App :  Dollhouse Decorating Games  Min SentimentP :  nan  Max SentimentP :  nan
App :  Dolphin Browser - Fast, Private & Adblock🐬  Min SentimentP :  nan  Max SentimentP :  nan
App :  Dolphins Live Wallpaper  Min SentimentP :  nan  Max SentimentP :  nan
App :  Domino's Pizza USA  Min SentimentP :  -0.3  Max SentimentP :  0.78125
App :  Dominos Game ✔️  Min SentimentP :  -0.8  Max SentimentP :  0.8
App :  Domofond Недвижимость. Купить, снять квартиру.  Min SentimentP

App :  FREE LIVE TALK  Min SentimentP :  nan  Max SentimentP :  nan
App :  FREE Stock Market Trading Tips  Min SentimentP :  nan  Max SentimentP :  nan
App :  FREEDOME VPN Unlimited anonymous Wifi Security  Min SentimentP :  0.0  Max SentimentP :  1.0
App :  FUN Keyboard – Emoji Keyboard, Sticker,Theme & GIF  Min SentimentP :  0.0  Max SentimentP :  0.5
App :  Fabulous: Motivate Me! Meditate, Relax, Sleep  Min SentimentP :  -0.07380952380952381  Max SentimentP :  0.8333333333333334
App :  Face Filter, Selfie Editor - Sweet Camera  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  Facebook  Min SentimentP :  -0.8  Max SentimentP :  0.7
App :  Facebook Ads Manager  Min SentimentP :  -0.5  Max SentimentP :  1.0
App :  Facebook Lite  Min SentimentP :  -1.0  Max SentimentP :  0.875
App :  Facebook Local  Min SentimentP :  -0.4318181818181818  Max SentimentP :  1.0
App :  Facebook Pages Manager  Min SentimentP :  -1.0  Max SentimentP :  0.7
App :  Facetune - Ad Free  Min SentimentP :  -0.

App :  Fruit Block - Puzzle Legend  Min SentimentP :  -0.18333333333333332  Max SentimentP :  -0.18333333333333332
App :  Fruit Boom  Min SentimentP :  nan  Max SentimentP :  nan
App :  Fruit Cube Blast  Min SentimentP :  nan  Max SentimentP :  nan
App :  Fruit Ninja®  Min SentimentP :  -1.0  Max SentimentP :  0.7
App :  Fruits Bomb  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  Fuelio: Gas log & costs  Min SentimentP :  -0.5  Max SentimentP :  1.0
App :  Full Screen Caller ID  Min SentimentP :  -0.20000000000000004  Max SentimentP :  1.0
App :  Fun Kid Racing  Min SentimentP :  -0.6499999999999999  Max SentimentP :  1.0
App :  Fun Kid Racing - Motocross  Min SentimentP :  -1.0  Max SentimentP :  1.0
App :  Funny Alarm Clock Ringtones  Min SentimentP :  -0.45  Max SentimentP :  0.7999999999999999
App :  Funny Pics  Min SentimentP :  -0.3125  Max SentimentP :  0.4400000000000001
App :  Funny Quotes Free  Min SentimentP :  nan  Max SentimentP :  nan
App :  Funny Ringtones  Min Sen

App :  Google Duo - High Quality Video Calls  Min SentimentP :  -0.30000000000000004  Max SentimentP :  0.8
App :  Google Earth  Min SentimentP :  -0.5000000000000001  Max SentimentP :  0.7333333333333334
App :  Google Fit - Fitness Tracking  Min SentimentP :  -0.5333333333333333  Max SentimentP :  0.875
App :  Google Handwriting Input  Min SentimentP :  -0.4  Max SentimentP :  1.0
App :  Google Keep  Min SentimentP :  -0.8  Max SentimentP :  1.0
App :  Google My Business  Min SentimentP :  -0.6  Max SentimentP :  1.0
App :  Google News  Min SentimentP :  -0.37083333333333335  Max SentimentP :  0.6333333333333333
App :  Google PDF Viewer  Min SentimentP :  -0.6  Max SentimentP :  0.725
App :  Google Pay  Min SentimentP :  -0.5229166666666666  Max SentimentP :  0.8
App :  Google Photos  Min SentimentP :  -0.5666666666666667  Max SentimentP :  0.75
App :  Google Play Books  Min SentimentP :  nan  Max SentimentP :  nan
App :  Google Play Games  Min SentimentP :  nan  Max SentimentP :  nan

FINE PARTE OBBLIGATORIA

In [304]:
#13 14
#For each app, compute the average number of words in its reviews
#For each app, compute its longest review
gpReview.Translated_Review = gpReview.Translated_Review.replace(np.nan, "")


In [309]:
for rap in gpReview.App.unique():
    gpRap = gpReview[gpReview.App==rap]
    word=0
    wordL=0
    maxWRev=gpRap.Translated_Review[0]
    rLen = len(gpRap.Translated_Review[0].split())
    for w in range(len(gpRap.Translated_Review)):
        wordL =len(gpRap.Translated_Review[w].split())
        word += wordL
        if wordL>rLen:
            maxWRev=gpRap.Translated_Review[w]
            rLen = wordL
    print(rap, " average number of words ",word/len(gpRap.Translated_Review))
    print(rap, " longest review: ", maxWRev)

10 Best Foods for You  average number of words  6.46
10 Best Foods for You  longest review:  WEALTH OF INFORMATION Very informative ... easy understand. (The ads middle page interupted read). Update: My "ad comment" NOT objection ads, literally position ads. Smack dab MIDDLE. I new spartphones & apps. Didnt know NOT Click eveything I saw... But, ... Full Review


KeyError: 0

In [298]:
word = 0
wordL = 0

maxWRev=gpRap.Translated_Review[0]
rLen = len(gpRap.Translated_Review[0].split())

for w in range(len(gpRap.Translated_Review)):
    wordL =len(gpRap.Translated_Review[w].split())
    word += wordL
    if wordL>rLen:
        maxWRev=gpRap.Translated_Review[w]
        rLen = wordL

KeyError: 0

In [315]:
gpRap

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
200,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,Great,Positive,0.800000,0.750000
201,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,nice,Positive,0.600000,1.000000
202,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,Almost mobile phone,Neutral,0.000000,0.000000
203,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,"Very effective, efficient convenient find avai...",Positive,0.626667,0.666667
204,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,Easy intuitive,Positive,0.433333,0.833333
205,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,good searching job internship,Positive,0.700000,0.600000
206,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,"Not bad, price little bit expensive",Negative,-0.112500,0.622222
207,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy use!,Positive,0.541667,0.833333
208,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,job search results iOS e.g. security,Neutral,0.000000,0.000000
209,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy,Positive,0.433333,0.833333


In [311]:
maxWRev=gpRap.Translated_Review[0]
rLen = len(gpRap.Translated_Review[0].split())

for w in range(len(gpRap.Translated_Review)):
    wordL =len(gpRap.Translated_Review[w].split())
    word += wordL
    if wordL>rLen:
        maxWRev=gpRap.Translated_Review[w]
        rLen = wordL

KeyError: 0

In [254]:
gpReview.Translated_Review = gpReview.Translated_Review.replace(np.nan, "")


In [271]:
 gpReview

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462
2,10 Best Foods for You,,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000
5,10 Best Foods for You,Best way,Positive,1.000000,0.300000
6,10 Best Foods for You,Amazing,Positive,0.600000,0.900000
7,10 Best Foods for You,,NaN,NaN,NaN
8,10 Best Foods for You,"Looking forward app,",Neutral,0.000000,0.000000
9,10 Best Foods for You,It helpful site ! It help foods get !,Neutral,0.000000,0.000000


In [20]:
#gpReview.Translated_Review.apply(lambda x : avgWord(x))
appR = gpReview.App.unique()
for r in 

array(['10 Best Foods for You', '104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室',
       '11st', ..., 'Hotwire Hotel & Car Rental App',
       'Housing-Real Estate & Property', 'Houzz Interior Design Ideas'],
      dtype=object)

print(len(x.split()))


def avgWord(y):
    

In [ ]:
gpApp['Android Ver']

In [170]:
maxG = ('b',0)
if (maxG[0] != 'a'):
    print (maxG)

('b', 0)
